<a href="https://colab.research.google.com/github/vveitch/causality-tutorials/blob/main/ATE_Estimation_with_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ATT Estimation Tutorial

This tutorial gives a short example for how to estimate average treatment effect on the treated using machine learning methods

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import sklearn
import os

In [2]:
RANDOM_SEED=42
np.random.seed(RANDOM_SEED)

## Load and Format LaLonde Observational Data

In [3]:
def make_data_lalonde(df):
    df_new = df.drop(['nodegree'], axis=1)
    df_new['pos74'] = (df_new['RE74'] > 0).astype(int)
    df_new['pos75'] = (df_new['RE75'] > 0).astype(int)
    df_new['treatment'] = df_new['treatment'].astype(int)
    return df_new


col_names = ['treatment', 'age', 'education', 'black',
             'hispanic', 'married', 'nodegree', 'RE74', 'RE75', 'RE78']
control = pd.read_csv('https://raw.githubusercontent.com/anishazaveri/austen_plots/master/data/imbens-raw/psid_controls.txt', header=None, sep=r"\s\s", names=col_names, engine='python')
treatment = pd.read_csv('https://raw.githubusercontent.com/anishazaveri/austen_plots/master/data/imbens-raw/nswre74_treated.txt', header=None, sep=r"\s\s", names=col_names, engine='python')

lalonde1 = pd.concat([control, treatment]).reset_index(drop=True)
lalonde1 = make_data_lalonde(lalonde1)

In [4]:
lalonde1.head()

,treatment,age,education,black,hispanic,married,RE74,RE75,RE78,pos74,pos75
0,0,47.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,0,50.0,12.0,1.0,0.0,1.0,0.0,0.0,0.0,0,0
2,0,44.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,0,28.0,12.0,1.0,0.0,1.0,0.0,0.0,0.0,0,0
4,0,54.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0


In [5]:
confounders = lalonde1.drop(columns=['RE78', 'treatment'])
outcome = lalonde1['RE78']
treatment = lalonde1['treatment']

## Specify Nuisance Function Models

The next step is to specify models for the conditional expected outcome and propensity score

In [6]:
# specify a model for the conditional expected outcome

# make a function that returns a sklearn model for later use in k-folding
def make_Q_model():
  return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)
Q_model = make_Q_model()

# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_w_treatment = confounders.copy()
X_w_treatment["treatment"] = treatment

X_train, X_test, y_train, y_test = train_test_split(X_w_treatment, outcome, test_size=0.2)
Q_model.fit(X_train, y_train)
y_pred = Q_model.predict(X_test)

test_mse=mean_squared_error(y_pred, y_test)
print(f"Test MSE of fit model {test_mse}") 
baseline_mse=mean_squared_error(y_train.mean()*np.ones_like(y_test), y_test)
print(f"Test MSE of no-covariate model {baseline_mse}")

Test MSE of fit model 105818236.45816422
Test MSE of no-covariate model 246319790.55062827


In [7]:
# specify a model for the propensity score

def make_g_model():
#  return LogisticRegression(max_iter=1000)
  return RandomForestClassifier(n_estimators=100, max_depth=5)

g_model = make_g_model()
# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_train, X_test, a_train, a_test = train_test_split(confounders, treatment, test_size=0.2)
g_model.fit(X_train, a_train)
a_pred = g_model.predict_proba(X_test)[:,1]

test_ce=log_loss(a_test, a_pred)
print(f"Test CE of fit model {test_ce}") 
baseline_ce=log_loss(a_test, a_train.mean()*np.ones_like(a_test))
print(f"Test CE of no-covariate model {baseline_ce}")

Test CE of fit model 0.07789407933364972
Test CE of no-covariate model 0.21817471356014154


## Use cross fitting to get get predicted outcomes and propensity scores for each unit

In [8]:
# helper functions to implement the cross fitting

def treatment_k_fold_fit_and_predict(make_model, X:pd.DataFrame, A:np.array, n_splits:int):
    """
    Implements K fold cross-fitting for the model predicting the treatment A. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns an array containing the predictions  

    Args:
    model: function that returns sklearn model (which implements fit and predict_prob)
    X: dataframe of variables to adjust for
    A: array of treatments
    n_splits: number of splits to use
    """
    predictions = np.full_like(A, np.nan, dtype=float)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    
    for train_index, test_index in kf.split(X, A):
      X_train = X.loc[train_index]
      A_train = A.loc[train_index]
      g = make_model()
      g.fit(X_train, A_train)

      # get predictions for split
      predictions[test_index] = g.predict_proba(X.loc[test_index])[:, 1]

    assert np.isnan(predictions).sum() == 0
    return predictions


def outcome_k_fold_fit_and_predict(make_model, X:pd.DataFrame, y:np.array, A:np.array, n_splits:int, output_type:str):
    """
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    y: array of outcomes
    A: array of treatments
    n_splits: number of splits to use
    output_type: type of outcome, "binary" or "continuous"

    """
    predictions0 = np.full_like(A, np.nan, dtype=float)
    predictions1 = np.full_like(y, np.nan, dtype=float)
    if output_type == 'binary':
      kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
      kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    # include the treatment as input feature
    X_w_treatment = X.copy()
    X_w_treatment["A"] = A

    # for predicting effect under treatment / control status for each data point 
    X0 = X_w_treatment.copy()
    X0["A"] = 0
    X1 = X_w_treatment.copy()
    X1["A"] = 1

    
    for train_index, test_index in kf.split(X_w_treatment, y):
      X_train = X_w_treatment.loc[train_index]
      y_train = y.loc[train_index]
      q = make_model()
      q.fit(X_train, y_train)

      if output_type =='binary':
        predictions0[test_index] = q.predict_proba(X0.loc[test_index])[:, 1]
        predictions1[test_index] = q.predict_proba(X1.loc[test_index])[:, 1]
      elif output_type == 'continuous':
        predictions0[test_index] = q.predict(X0.loc[test_index])
        predictions1[test_index] = q.predict(X1.loc[test_index])

    assert np.isnan(predictions0).sum() == 0
    assert np.isnan(predictions1).sum() == 0
    return predictions0, predictions1

In [9]:
g = treatment_k_fold_fit_and_predict(make_g_model, X=confounders, A=treatment, n_splits=10)

In [10]:
Q0,Q1=outcome_k_fold_fit_and_predict(make_Q_model, X=confounders, y=outcome, A=treatment, n_splits=10, output_type="continuous")

In [11]:
data_and_nuisance_estimates = pd.DataFrame({'g': g, 'Q0': Q0, 'Q1': Q1, 'A': treatment, 'Y': outcome})
data_and_nuisance_estimates.head()

,g,Q0,Q1,A,Y
0,0.313350,91.666309,1612.832142,0,0.0
1,0.191958,2017.511419,3896.496922,0,0.0
2,0.470788,43.925636,1767.068998,0,0.0
3,0.517957,11169.331018,9051.689591,0,0.0
4,0.014246,0.000000,2117.226099,0,0.0


## Combine predicted values and data into estimate of ATT

In [12]:
def att_aiptw(Q0, Q1, g, A, Y, prob_t=None):
  """
  # Double ML estimator for the ATT
  This uses the ATT specific scores, see equation 3.9 of https://www.econstor.eu/bitstream/10419/149795/1/869216953.pdf
  """

  if prob_t is None:
    prob_t = A.mean() # estimate marginal probability of treatment

  tau_hat = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0)).mean()/ prob_t
  
  scores = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0) - tau_hat*A) / prob_t
  n = Y.shape[0] # number of observations
  std_hat = np.std(scores) / np.sqrt(n)

  return tau_hat, std_hat


In [13]:
def ate_aiptw(Q0, Q1, g, A, Y, prob_t=None):
  """
  # Double ML estimator for the ATE
  """

  tau_hat = (Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g)).mean()
  
  scores = Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g) - tau_hat
  n = Y.shape[0] # number of observations
  std_hat = np.std(scores) / np.sqrt(n)

  return tau_hat, std_hat


In [14]:
tau_hat, std_hat = att_aiptw(**data_and_nuisance_estimates)
print(f"The estimate is {tau_hat} pm {1.96*std_hat}")

The estimate is 1295.9085019166787 pm 1624.0222463376508


In [15]:
in_treated = data_and_nuisance_estimates['A']==1
treated_estimates = data_and_nuisance_estimates[in_treated]
tau_hat, std_hat = ate_aiptw(**treated_estimates)
print(f"The estimate is {tau_hat} pm {1.96*std_hat}")

The estimate is -34006.316596350116 pm 51141.073469511226


In [16]:
# The LaLonde data has severe overlap issues. Lets try computing the estimate restricted to a population with only reasonable propensity scores
g = data_and_nuisance_estimates['g']
in_overlap_popluation = (g < 0.90)
overlap_data_and_nuisance = data_and_nuisance_estimates[in_overlap_popluation]
tau_hat, std_hat = att_aiptw(**overlap_data_and_nuisance)
print(f"The estimate is {tau_hat} pm {1.96*std_hat}")

The estimate is 563.251470303294 pm 1502.651264797957


## Question 3 c)

In [17]:
def att_plug_in(Q0, Q1, Y):
  """
  Simple plug-in estimator for the ATT

  args:
  Q0: predictions for treated units conditional on A=0 (untreated)
  Q1: predictions for treated units conditional on A=1 (treated)
  Y: array of outcomes for treated units
  """
  # compute the mean of F(1, X) - F(0, X) for only those individuals who received treatment
  tau_hat = (Q1 - Q0).mean()
  
  scores = (Q1 - Q0) - tau_hat
  n = Y.shape[0] # number of observations
  std_hat = np.std(scores) / np.sqrt(n)

  return tau_hat, std_hat

In [18]:
# Only choose samples who received the treatment
in_treated = data_and_nuisance_estimates['A'] == 1
treated_estimates = data_and_nuisance_estimates[in_treated][["Q0", "Q1", "Y"]]

# Pass the samples and their predictions to the ATT function
tau_ATT, std_ATT = att_plug_in(**treated_estimates)

print(f"The estimate of τ^(ATT) with the plug in estimator is {tau_ATT}")

# Compare with the estimator from above with the "reasonable" propensity score
print(f"The estimate of τ^(ATT) with the advanced ML approach is {tau_hat}")

The estimate of τ^(ATT) with the plug in estimator is 1085.3404537748204
The estimate of τ^(ATT) with the advanced ML approach is 563.251470303294


We can observe that our estimator is not very close to the advanced estimator, being almost twice the quantity of the latter.

However this may be explained by the fact that the plug in estimator does not take into account the effect of the nuisance parameters specified in the model as well as the propensity scores

## Question 3 d)

In [19]:
# Number of bootstrap samples
num_bootstrap_samples = 1000

# Function to compute τ^{ATT} for each bootstrap sample (with replacement)
def compute_tau_att_bootstrap_sample(data_and_nuisance_estimates):
    bootstrap_sample = data_and_nuisance_estimates.sample(frac=1, replace=True)

    in_treated = bootstrap_sample['A'] == 1
    treated_estimates = bootstrap_sample[in_treated][["Q0", "Q1", "Y"]]
    
    tau_ATT, _ = att_plug_in(**treated_estimates)
    return tau_ATT

# Perform bootstrap
bootstrap_samples = [compute_tau_att_bootstrap_sample(data_and_nuisance_estimates) for _ in range(num_bootstrap_samples)]

# Calculate 95% confidence interval
confidence_interval = np.percentile(bootstrap_samples, [2.5, 97.5])
print(f"95% Confidence Interval for estimate of τ^(ATT) using the bootstrap: {list(confidence_interval)}")

# Compare against the confidence interval for the estimator from above with the "reasonable" propensity score
print(f"95% Confidence Interval for estimate of τ^(ATT) with the advanced ML approach: {[tau_hat - std_hat * 1.96, tau_hat + std_hat * 1.96]}")

95% Confidence Interval for estimate of τ^(ATT) using the bootstrap: [669.2862120150093, 1506.8020959546695]
95% Confidence Interval for estimate of τ^(ATT) with the advanced ML approach: [-939.3997944946631, 2065.902735101251]


Once again we observe that the confidence intervals for the two approaches differ, and this can be explained for the same reasons as in 3 c).  
However, one can note that the confidence interval for the bootstrap estimator is wholly contained within the confidence interval for the more advanced approach that takes into account more factors than our simple estimator.